More complex format
===================

* xml
* html

XML
===
    * data transfer
    * nested elements
    * document validation
    * parsers in most languages
    * focus on bussines
    * 

```xml
<article>
    <info>
        <title>Some title</title>
        <authors>
            <author>
                <firstname>Jakub</firstname>
                <lastname>Kanclerz</lastname>
            </author>
        </authors>
    </info>
    <content>
        <section type="header">
            <paragraph>
                some title
            </paragraph>
        </section>
        <section type="main">
            <paragraph>
                some content
            </paragraph>
            <image src="image/src.jpeg"/>
        </section>
    </content>
</article>
```

Html
====

```html
<li class="link">
    <div class="article" data-id="4789115">
        <a href="#4789115">
            <span>85</span>
            <span class="button button submit">wykop</span>
        </a>
    </div>

    <div class="media-content m-reset-float ">
        <a href="https://www.wykop.pl/link/4789115/w-jaki-sposob-facebook-stal-sie-lewicowa-sekta-opowiesc-pracownika-firmy/">
            <img src="https://www.wykop.pl/cdn/c3397993/link_MwUxfsmJdpfvuiy0ICl1l0Yw35VeWgBc,w207h139.jpg" alt="W jaki sposób Facebook stał się lewicową sektą? Opowieść pracownika firmy.">
        </a>
    </div>
    <div class="lcontrast m-reset-margin">
        <h2>
            <a href="https://www.wykop.pl/link/4789115/w-jaki-sposob-facebook-stal-sie-lewicowa-sekta-opowiesc-pracownika-firmy/"
                rel="nofollow noopener noreferrer" title="W jaki sposób Facebook stał się lewicową sektą? Opowieść pracownika firmy.">W
                jaki sposób Facebook stał się lewicową sektą? Opowieść pracownika firmy.</a>
        </h2>


        <div class="description">
            <p class="text">
                <a href="https://www.wykop.pl/link/4789115/w-jaki-sposob-facebook-stal-sie-lewicowa-sekta-opowiesc-pracownika-firmy/"
                    title="">
                    Tragiczna relacja pracownika, który przez lata obserwował jak serwis promujący wolność słowa
                    zamienił się w ideologiczny rynsztok prześladujący pracowników o zróżnicowanym światopoglądzie.
                </a>
            </p>
        </div>
    </div>
</li>
```

In [1]:
!pip install bs4

     |████████████████████████████████| 112kB 661kB/s eta 0:00:01
    Running setup.py install for bs4 ... done
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
sample_xml = '''
<article>
    <info>
        <title>Some title</title>
        <authors>
            <author>
                <firstname>Jakub</firstname>
                <lastname>Kanclerz</lastname>
            </author>
        </authors>
    </info>
    <content>
        <section type="header">
            <paragraph>
                some title
            </paragraph>
        </section>
        <section type="main">
            <paragraph>
                some content
            </paragraph>
            <image src="image/src.jpeg"/>
        </section>
    </content>
</article>
'''

In [6]:
import requests
from bs4 import BeautifulSoup as bs
from itertools import chain

In [7]:
!pip install lxml

In [10]:
loaded = bs(sample_xml, "html.parser")

In [11]:
urls = [
    "https://www.wykop.pl/strona/{}".format(page)
    for page in range(1, 10, 1)
]

In [12]:
import collections
from pprint import pprint as pp

Wykop = collections.namedtuple('Wykop', ['count', 'title', 'description', 'tags'])

def clear_text(text):
    return text.rstrip().replace("\n", '').replace("\t", '')

def grab_wykop_item_data(element):
    try:
        tags = map(lambda x: x.text, element.select('.article>div a.tag'))
        tags = filter(lambda x: x.startswith('#'), tags)
        
        return Wykop(
            int(element.select('.article>div a>span:first-child')[0].text),
            element.select('.article>div h2 a')[0]['title'],
            clear_text(element.select('.article>div .description')[0].text),
            list(tags))
    except Exception as e:
        return None

In [13]:
pages = (requests.get(url) for url in urls)
contents = (page.content for page in pages)
soups = (bs(content, 'html.parser') for content in contents)
lists_of_items = (soup.select('#itemsStream li.iC') for soup in soups)
all_items_together = chain.from_iterable(lists_of_items)
extracted = (grab_wykop_item_data(item) for item in all_items_together)
valid = (wykop for wykop in extracted if wykop is not None)

In [14]:
import itertools
first5 = [next(valid) for x in range(4)]

In [15]:
list(first5)

[Wykop(count=325, title='Trwa spontaniczny protest branży fitness w Warszawie.', description='"100 tysięcy miejsc pracy jest na szali"', tags=['#polska', '#ekonomia', '#silownia', '#mikrokoksy', '#koronawirus']),
 Wykop(count=112, title='Braun zdjęty z wizji w „Forum” TVP Info. Promował książę o fałszywości pandemii', description='Grzegorz Braun kilkanaście minut po starcie programu „Forum” w TVP Info zakończył w nim udział. Prowadzący Adrian Klarenbach bezskutecznie prosił go, żeby nie eksponował do kamery okładki książki „Fałszywa pandemia”, bo to lokowanie produktu niedozwolone w programie publicystycznym.', tags=['#polityka', '#polska', '#wydarzenia', '#koronawirus', '#tvpis', '#tvpiscodzienny']),
 Wykop(count=1952, title='KOLEJNY REKORD! 9622 nowych przypadków koronawirusa, 84 zgonów', description='Ministerstwo Zdrowia w sobotnim komunikacie poinformowało o 9622 nowych i potwierdzonych przypadkach koronawirusa. To niestety kolejny rekordowy wynik zakażeń w Polsce. Zmarło również 8

In [16]:
tags = (wykop.tags for wykop in first5)

In [17]:
all_tags = list(chain.from_iterable(tags))

In [18]:
counter = {}

for tag in all_tags:
    if tag not in counter.keys():
        counter[tag] = 1
    else:
        counter[tag] = counter[tag] + 1

In [19]:
counter

{'#polska': 3,
 '#ekonomia': 1,
 '#silownia': 1,
 '#mikrokoksy': 1,
 '#koronawirus': 4,
 '#polityka': 1,
 '#wydarzenia': 1,
 '#tvpis': 1,
 '#tvpiscodzienny': 1,
 '#krakow': 1,
 '#covid19': 1,
 '#wirus': 1,
 '#bekazpisu': 1,
 '#nauka': 1,
 '#ksiazki': 1,
 '#bekazkatoli': 1,
 '#medycyna': 1,
 '#ciekawostki': 1}

In [20]:
import operator
list(reversed(sorted(counter.items(), key=lambda item: item[1])))

[('#koronawirus', 4),
 ('#polska', 3),
 ('#ciekawostki', 1),
 ('#medycyna', 1),
 ('#bekazkatoli', 1),
 ('#ksiazki', 1),
 ('#nauka', 1),
 ('#bekazpisu', 1),
 ('#wirus', 1),
 ('#covid19', 1),
 ('#krakow', 1),
 ('#tvpiscodzienny', 1),
 ('#tvpis', 1),
 ('#wydarzenia', 1),
 ('#polityka', 1),
 ('#mikrokoksy', 1),
 ('#silownia', 1),
 ('#ekonomia', 1)]

In [21]:
from collections import Counter
c = Counter(all_tags)
c.most_common()

[('#koronawirus', 4),
 ('#polska', 3),
 ('#ekonomia', 1),
 ('#silownia', 1),
 ('#mikrokoksy', 1),
 ('#polityka', 1),
 ('#wydarzenia', 1),
 ('#tvpis', 1),
 ('#tvpiscodzienny', 1),
 ('#krakow', 1),
 ('#covid19', 1),
 ('#wirus', 1),
 ('#bekazpisu', 1),
 ('#nauka', 1),
 ('#ksiazki', 1),
 ('#bekazkatoli', 1),
 ('#medycyna', 1),
 ('#ciekawostki', 1)]

persistance
-------------

In [221]:
import csv
with open('wykop.csv', 'w') as destination_file:
    writer = csv.writer(destination_file, delimiter=';', quotechar='"')
    for row in first5:
        writer.writerow(row)

In [226]:
import json
with open('wykop.data', 'w') as destination_file:
    for row in first5:
        destination_file.write(json.dumps(row._asdict()) + "\n")